##1. Download Data

In [2]:
!curl -X GET https://raw.githubusercontent.com/tensorflow/nmt/master/nmt/scripts/download_iwslt15.sh -o download_iwslt15.sh
!chmod +x download_iwslt15.sh && ./download_iwslt15.sh nmt_data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1188  100  1188    0     0   7867      0 --:--:-- --:--:-- --:--:--  7867
mkdir: created directory 'nmt_data'
Download training dataset train.en and train.vi.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.9M  100 12.9M    0     0  9782k      0  0:00:01  0:00:01 --:--:-- 9782k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.2M  100 17.2M    0     0  13.3M      0  0:00:01  0:00:01 --:--:-- 13.3M
Download dev dataset tst2012.en and tst2012.vi.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Spee

In [3]:
!ls -l

total 12
-rwxr-xr-x 1 root root 1188 Jan 18 13:22 download_iwslt15.sh
drwxr-xr-x 2 root root 4096 Jan 18 13:22 nmt_data
drwxr-xr-x 1 root root 4096 Jan  8 17:15 sample_data


## 2. Clone code

In [4]:
!git clone https://github.com/tensorflow/nmt.git

Cloning into 'nmt'...
remote: Enumerating objects: 1247, done.
remote: Total 1247 (delta 0), reused 0 (delta 0), pack-reused 1247
Receiving objects: 100% (1247/1247), 1.22 MiB | 8.67 MiB/s, done.
Resolving deltas: 100% (892/892), done.


In [5]:
!ls -l nmt/nmt

total 236
-rw-r--r-- 1 root root  7387 Jan 18 13:23 attention_model.py
drwxr-xr-x 3 root root  4096 Jan 18 13:23 g3doc
-rw-r--r-- 1 root root 12252 Jan 18 13:23 gnmt_model.py
-rw-r--r-- 1 root root  8895 Jan 18 13:23 inference.py
-rw-r--r-- 1 root root  6490 Jan 18 13:23 inference_test.py
-rw-r--r-- 1 root root     0 Jan 18 13:23 __init__.py
-rw-r--r-- 1 root root 24395 Jan 18 13:23 model_helper.py
-rw-r--r-- 1 root root 33588 Jan 18 13:23 model.py
-rw-r--r-- 1 root root 48465 Jan 18 13:23 model_test.py
-rw-r--r-- 1 root root 29123 Jan 18 13:23 nmt.py
-rw-r--r-- 1 root root  3404 Jan 18 13:23 nmt_test.py
drwxr-xr-x 2 root root  4096 Jan 18 13:23 scripts
drwxr-xr-x 2 root root  4096 Jan 18 13:23 standard_hparams
drwxr-xr-x 2 root root  4096 Jan 18 13:23 testdata
-rw-r--r-- 1 root root 28948 Jan 18 13:23 train.py
drwxr-xr-x 2 root root  4096 Jan 18 13:23 utils


## 3. Tensor Board

### Download

In [6]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!rm -rf ngrok
!unzip ngrok-stable-linux-amd64.zip

--2019-01-18 13:23:16--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.226.180.131, 34.206.9.96, 34.206.36.121, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.226.180.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  20.1MB/s    in 0.3s    

2019-01-18 13:23:17 (20.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


## 4. Training

### 4.1 Basic

In [0]:
tsboard_cmd = 'tensorboard --logdir nmt_model --host 0.0.0.0 --port 6006 &'
print("Log dir", tsboard_cmd)
get_ipython().system_raw(tsboard_cmd)
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
!rm -rf nmt_model 
!mkdir nmt_model
!python -m nmt.nmt.nmt \
    --src=vi --tgt=en \
    --vocab_prefix=nmt_data/vocab  \
    --train_prefix=nmt_data/train \
    --dev_prefix=nmt_data/tst2012  \
    --test_prefix=nmt_data/tst2013 \
    --out_dir=nmt_model \
    --num_train_steps=12000 \
    --steps_per_stats=100 \
    --num_layers=4 \
    --num_units=128 \
    --dropout=0.2 \
    --metrics=bleu

In [0]:
!ls -l nmt_model

### 4.2 Attention

In [0]:
# Clean old model
!rm -rf nmt_attention_model
!mkdir nmt_attention_model

In [8]:
tsboard_cmd = 'tensorboard --logdir nmt_attention_model --host 0.0.0.0 --port 6006 &'
print("Log dir", tsboard_cmd)
get_ipython().system_raw(tsboard_cmd)
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Log dir tensorboard --logdir nmt_attention_model --host 0.0.0.0 --port 6006 &
http://2aeec91b.ngrok.io


In [9]:
!python -m nmt.nmt.nmt \
    --attention=scaled_luong \
    --src=vi --tgt=en \
    --vocab_prefix=nmt_data/vocab  \
    --train_prefix=nmt_data/train \
    --dev_prefix=nmt_data/tst2012  \
    --test_prefix=nmt_data/tst2013 \
    --out_dir=nmt_attention_model \
    --num_train_steps=24000 \
    --steps_per_stats=100 \
    --infer_mode=beam_search\
    --beam_width=10\
    --num_layers=4 \
    --num_units=128 \
    --dropout=0.2 \
    --metrics=bleu 
#     --encoder_type=gnmt\
#     --num_encoder_layers=8\
#     --num_decoder_layers=8
#     --decay_scheme=luong10

# Job id 0
2019-01-18 13:23:45.678756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-01-18 13:23:45.679512: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2019-01-18 13:23:45.679577: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1511] Adding visible gpu devices: 0
2019-01-18 13:23:46.611541: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-01-18 13:23:46.611608: I tensorflow/core/common_runtime/gpu/gpu_device.cc:988]      0 
2019-01-18 13:23:46.611625: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1001] 0:   N 
2019-01-18 13:23:46.611882: W tensorflow/core/common_runtime/gpu/gpu_bfc_a